## Entraînement du modèle SOMME_RNN_TEST

Pour tester la validité de l'implémentation des RNN, et donc s'assurer que les prédictions constantes du modèle DIST_REL_C_RNN ne sont pas dues à une mauvaise création ou gestion des données en entrée, nous allons créer un modèle dont le but est de prédire la somme d'un certain nombre d'entrées.

Le format des tableaux d'entrée est identique à celui du modèle DIST_REL_C_RNN, et nous allons utiliser un réseau de neurones très similaire pour effectuer les prédictions. Les données d'entrée et de cible ont été créées dans le notebook 10.0.

#### Chemin des fichiers

In [7]:
train_prepared_input_loc = "../data/SOMME_RNN_TEST/train_prepared_input.h5"
train_labels_loc = "../data/SOMME_RNN_TEST/train_labels.h5"

minimal_prepared_input_loc = "../data/SOMME_RNN_TEST/minimal_prepared_input.h5"
minimal_labels_loc = "../data/SOMME_RNN_TEST/minimal_labels.h5"

models_loc = "../models/SOMME_RNN_TEST/10.0/"
logs_loc = "../models/SOMME_RNN_TEST/10.0/"

### Création du RN

#### Fonction de coût (RMSE)

In [8]:
def rmse(pred, targets):
    with tf.name_scope("rmse_loss"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, targets)), name="rmse")

#### Fonction d'évaluation des performances (opposé du RMSE)

In [9]:
def rmse_valid(pred, targets, inputs):
    with tf.name_scope("rmse_validation"):
        return -rmse(pred, targets)

#### Fonction de création du RN

Cette fonction est largement inspirée de l'exemple suivant : https://github.com/tflearn/tflearn/blob/master/examples/images/rnn_pixels.py

In [10]:
import tflearn as tfl


def creer_RNN(batch_size, activation="tanh", inner_activation="sigmoid", outlayer_activation="linear",
              depth=2, width=58, dropout_val=0.98, learning_rate=0.001):
    
    network = tfl.input_data(shape=[None, 870, 1], name="input")
    
    for i in range(depth):
        network = tfl.lstm(network, width, return_seq=(i != depth-1), name="hl"+str(i))
        network = tfl.dropout(network, dropout_val)
        
    network = tfl.fully_connected(network, 1, activation='linear')
    
    network = tfl.regression(network, optimizer='adam', loss=rmse, metric=rmse_valid, name="outlayer", 
                             learning_rate=learning_rate)
    
    return network
    

## Entraînement du modèle

In [11]:
import tensorflow as tf

def train_DIST_REL_C_RNN(X, y, model_name, model_path, logs_path, batch_size=1000, epochs=2, 
                         learning_rate=0.001, dropout_val=0.98, act="tanh", inner_act="sigmoid",
                         outlayer_act="linear", depth=2, width=58):
    
    tf.reset_default_graph()
        
    # Création du réseau
    network = creer_RNN(batch_size, activation=act, inner_activation=inner_act, outlayer_activation=outlayer_act,
                        depth=depth, width=width, dropout_val=dropout_val, learning_rate=learning_rate)
    
    # Création du modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)
    
    # Entraînement
    model.fit(X_inputs=X,Y_targets=y, batch_size=batch_size,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name, n_epoch=epochs)
    
    
    # Sauvegarde du modèle
    model.save(model_path+model_name+".tfl")


In [13]:
import h5py
import numpy as np
import tensorflow as tf
from tflearn.data_utils import pad_sequences


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')
"""
input_X_h5 = h5py.File(minimal_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(minimal_labels_loc, 'r')
"""

input_X = np.array(input_X_h5["inputs"])

input_X = pad_sequences(input_X, dtype="float32", maxlen=870)

input_X = input_X.reshape(-1, 870, 1)

labels_y = np.array(labels_y_h5["targets"])
labels_y = labels_y.reshape(-1, 1)


model_name = "SOMME_RNN_TEST_basic"

model_path = models_loc
logs_path = logs_loc

train_DIST_REL_C_RNN(input_X, labels_y, model_name, model_path, logs_path, batch_size=5000, 
                     epochs=100, learning_rate=0.05, dropout_val=0.99, depth=1, width=50)


Training Step: 123  | total loss: 2419.36328 | time: 1.204s
| Adam | epoch: 014 | loss: 2419.36328 - rmse_validation/Neg: -2419.3633 -- iter: 6000/9000


KeyboardInterrupt: 